In [1]:
import tkinter as tk
from tkinter import messagebox
import random
from PIL import Image, ImageTk

# Word categories with more words
WORDS = {
    "Fruits": ["APPLE", "BANANA", "MANGO", "ORANGE", "PINEAPPLE", "WATERMELON", "GRAPES", "STRAWBERRY"],
    "Vegetables": ["CARROT", "POTATO", "SPINACH", "TOMATO", "ONION", "BROCCOLI", "CUCUMBER", "PEAS"],
    "Months": ["JANUARY", "FEBRUARY", "MARCH", "APRIL", "AUGUST", "SEPTEMBER", "DECEMBER"],
    "Days": ["MONDAY", "TUESDAY", "WEDNESDAY", "FRIDAY", "SUNDAY", "THURSDAY"],
    "Countries": ["INDIA", "CANADA", "BRAZIL", "FRANCE", "JAPAN", "GERMANY", "AUSTRALIA", "MEXICO"],
    "Animals": ["ELEPHANT", "GIRAFFE", "KANGAROO", "DOLPHIN", "CHEETAH", "PENGUIN"]
}

# Colors
BG_COLOR = "#f0f8ff"
BUTTON_COLOR = "#e6e6fa"
TEXT_COLOR = "#333333"
CORRECT_COLOR = "#4caf50"
WRONG_COLOR = "#f44336"

# Initialize window
root = tk.Tk()
root.title("Hangman Game")
root.configure(bg=BG_COLOR)
root.geometry("800x650")

# Game state
category = ""
word = ""
guessed = []
wrong_guesses = 0
score = 0
high_score = 0

# Hangman images (we'll use drawings instead of actual images)
def create_hangman_images():
    images = []
    for i in range(7):
        img = tk.PhotoImage(width=300, height=300)
        images.append(img)
    return images

hangman_images = create_hangman_images()

# Canvas for hangman drawing with better styling
canvas = tk.Canvas(root, width=350, height=350, bg="white", highlightthickness=0)
canvas.grid(row=0, column=0, columnspan=10, pady=10, padx=10)

# GUI text labels with better styling
word_label = tk.StringVar()
hint_label = tk.StringVar()
score_label = tk.StringVar(value="Score: 0")
high_score_label = tk.StringVar(value="High Score: 0")

tk.Label(root, textvariable=word_label, font=("Comic Sans MS", 28), bg=BG_COLOR, fg=TEXT_COLOR).grid(row=1, column=0, columnspan=10, pady=10)
tk.Label(root, textvariable=hint_label, font=("Arial", 14, "italic"), bg=BG_COLOR, fg="#555555").grid(row=2, column=0, columnspan=10)
tk.Label(root, textvariable=score_label, font=("Arial", 12, "bold"), bg=BG_COLOR, fg="#0066cc").grid(row=3, column=0, columnspan=5)
tk.Label(root, textvariable=high_score_label, font=("Arial", 12, "bold"), bg=BG_COLOR, fg="#ff6600").grid(row=3, column=5, columnspan=5)

# Draw hangman with better graphics
def draw_gallows():
    canvas.delete("all")
    # Draw nice gallows with some styling
    canvas.create_line(50, 280, 220, 280, width=4, fill="#8b4513")  # Base
    canvas.create_line(100, 280, 100, 50, width=4, fill="#8b4513")  # Pole
    canvas.create_line(100, 50, 200, 50, width=4, fill="#8b4513")   # Top
    canvas.create_line(200, 50, 200, 80, width=3, fill="#8b4513")   # Rope

# Draw body parts with better visuals
def draw_parts(wrong):
    if wrong >= 1:  # Head
        canvas.create_oval(170, 80, 230, 140, width=3, outline="#333333")
    if wrong >= 2:  # Body
        canvas.create_line(200, 140, 200, 220, width=3, fill="#333333")
    if wrong >= 3:  # Left arm
        canvas.create_line(200, 160, 160, 190, width=3, fill="#333333")
    if wrong >= 4:  # Right arm
        canvas.create_line(200, 160, 240, 190, width=3, fill="#333333")
    if wrong >= 5:  # Left leg
        canvas.create_line(200, 220, 170, 260, width=3, fill="#333333")
    if wrong >= 6:  # Right leg
        canvas.create_line(200, 220, 230, 260, width=3, fill="#333333")
    if wrong >= 7:  # Sad face (game over)
        canvas.create_line(180, 100, 190, 110, width=2, fill="#333333")  # Left eye
        canvas.create_line(210, 100, 220, 110, width=2, fill="#333333")  # Right eye
        canvas.create_arc(180, 110, 220, 130, start=0, extent=-180, width=2, style=tk.ARC, outline="#333333")  # Frown

# Update word display with better styling
def update_word():
    display = ""
    for letter in word:
        if letter in guessed:
            display += letter + " "
        else:
            display += "_ "
    word_label.set(display)

# Check for win/lose with visual feedback
def check_game_over():
    global score, high_score
    if all(letter in guessed for letter in word):
        word_label.set("YOU WIN! 🎉")
        score += 10
        score_label.set(f"Score: {score}")
        if score > high_score:
            high_score = score
            high_score_label.set(f"High Score: {high_score}")
        disable_buttons()
        canvas.create_text(150, 150, text="WINNER!", font=("Arial", 36, "bold"), fill=CORRECT_COLOR)
        messagebox.showinfo("Congratulations", f"You won!\nThe word was: {word}\nYour score: {score}")
    elif wrong_guesses >= 6:
        word_label.set("GAME OVER 💀")
        disable_buttons()
        draw_parts(7)  # Draw sad face
        canvas.create_text(150, 150, text="GAME OVER", font=("Arial", 30, "bold"), fill=WRONG_COLOR)
        messagebox.showinfo("Game Over", f"The word was: {word}\nYour score: {score}")

# Handle letter guess with visual feedback
def guess_letter(letter):
    global wrong_guesses, score
    buttons[letter]['state'] = 'disabled'
    
    if letter in word:
        buttons[letter].config(bg=CORRECT_COLOR, fg="white")
        guessed.append(letter)
        score += 1
        score_label.set(f"Score: {score}")
    else:
        buttons[letter].config(bg=WRONG_COLOR, fg="white")
        wrong_guesses += 1
        draw_parts(wrong_guesses)
        if score > 0:  # Don't go below 0
            score -= 1
            score_label.set(f"Score: {score}")
    
    update_word()
    check_game_over()

# Disable all letter buttons
def disable_buttons():
    for btn in buttons.values():
        btn['state'] = 'disabled'

# Restart game with visual reset
def restart_game():
    global word, category, guessed, wrong_guesses, score
    category = random.choice(list(WORDS.keys()))
    word = random.choice(WORDS[category])
    guessed = []
    wrong_guesses = 0
    score = 0
    score_label.set(f"Score: {score}")
    draw_gallows()
    update_word()
    hint_label.set(f"Category: {category}")
    for btn in buttons.values():
        btn.config(state='normal', bg=BUTTON_COLOR, fg=TEXT_COLOR)

# Create letter buttons with better styling
buttons = {}
letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
for index, letter in enumerate(letters):
    btn = tk.Button(
        root, 
        text=letter, 
        width=3, 
        font=("Arial", 12, "bold"),
        bg=BUTTON_COLOR,
        fg=TEXT_COLOR,
        activebackground="#dcd0ff",
        relief=tk.RAISED,
        borderwidth=2,
        command=lambda l=letter: guess_letter(l)
    )
    row = 4 + index // 9
    col = index % 9
    btn.grid(row=row, column=col, padx=2, pady=2)
    buttons[letter] = btn

# Control buttons with better styling
button_frame = tk.Frame(root, bg=BG_COLOR)
button_frame.grid(row=10, column=0, columnspan=10, pady=15)

tk.Button(
    button_frame, 
    text="New Game", 
    width=12, 
    font=("Arial", 12, "bold"),
    bg="#4caf50",
    fg="white",
    activebackground="#388e3c",
    command=restart_game
).pack(side=tk.LEFT, padx=10)

tk.Button(
    button_frame, 
    text="Exit", 
    width=12, 
    font=("Arial", 12, "bold"),
    bg="#f44336",
    fg="white",
    activebackground="#d32f2f",
    command=root.destroy
).pack(side=tk.LEFT, padx=10)

# Start game with a welcome message
def show_welcome():
    canvas.delete("all")
    canvas.create_text(
        150, 150, 
        text="HANGMAN", 
        font=("Arial", 36, "bold"), 
        fill="#0066cc"
    )
    canvas.create_text(
        150, 200, 
        text="Click New Game to Start!", 
        font=("Arial", 14), 
        fill="#555555"
    )
    messagebox.showinfo("Welcome", "Welcome to Hangman!\nTry to guess the word before the man gets hanged.")

show_welcome()
root.mainloop()
